#### 설계:전주

In [1]:
import os
import warnings
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# 사용자정의 함수: key를 입력하면 해당 데이터를 데이터프레임으로 읽어옴
from aiddd.data_manager import read_data, write_data
from freeman.plt_setting import plt_settings

# matplotlib 설정: 한글표시, 차트 레이블 '-' 처리
plt_settings()

In [2]:
df_node = read_data('설계 전주')
df_node.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8458 entries, 0 to 8457
Data columns (total 62 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   공사번호             8458 non-null   int64  
 1   공사변경순번           8458 non-null   int64  
 2   전산화번호            8458 non-null   object 
 3   GISID            8458 non-null   int64  
 4   전주순번             8458 non-null   int64  
 5   공사구분코드           8458 non-null   object 
 6   공사성격코드           8458 non-null   object 
 7   전주형태코드           8458 non-null   object 
 8   전주종류코드           8399 non-null   object 
 9   전주규격코드           8399 non-null   float64
 10  전주수량             8458 non-null   int64  
 11  도색판공사특성코드        4612 non-null   object 
 12  도색판종류코드          8458 non-null   object 
 13  전주버팀목공사특성코드      8129 non-null   object 
 14  전주버팀목수량          8458 non-null   int64  
 15  번호판구분코드          8392 non-null   object 
 16  철주철탑중량           8458 non-null   int64  
 17  각입방법코드        

In [3]:
df_node['공사번호'].value_counts().sort_values(ascending=False)

공사번호
442720194689    83
442720203885    53
442720193507    41
442720204021    34
442720194067    26
                ..
442720233593     1
442720233691     1
442720233666     1
442720233664     1
442720204550     1
Name: count, Length: 3886, dtype: int64

* 전처리 없이 일단 전선 갯 수만 확인

In [5]:
# 공사비 데이터 읽어오기
df_cc = read_data('전처리 공사비+설계')

In [6]:
# 설계:전주에서 공사번호 중복 제거
unique_no_line = df_node['공사번호'].unique()
len(unique_no_line)

3886

In [7]:
# 공사비의 공사번호가 있는 설계:전주 레코드 추출
df_node_cc = df_node[df_node['공사번호'].isin(df_cc['공사번호'])]
len(df_node_cc)

6927

In [8]:
# 설계:전주에서 공사번호별 레코드 갯 수(공사번호별 전주 갯 수) 추출
sr_node_cnt = df_node['공사번호'].value_counts()
sr_node_cnt.sum(), df_node.shape

(8458, (8458, 62))

In [9]:
# 공사비 데이터프레임에 전주수 컬럼 추가
df_cc = pd.merge(
    df_cc, sr_node_cnt.rename('전주수'),
    left_on='공사번호', right_on=sr_node_cnt.index, how='left'
)
df_cc['전주수'] = df_cc['전주수'].fillna(0)
df_cc['전주수'].sum()

6927.0

In [10]:
write_data('전처리 공사비+설계', df_cc)